In [ ]:
import pandas as pd

In [106]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
#give the input text to the variable gloss_text
gloss_text = "all feel started"
tokens = nltk.word_tokenize(gloss_text)
lemmatizer = WordNetLemmatizer()
gloss_vals = []
# Iterate over each token
for token in tokens:
  lemma = lemmatizer.lemmatize(token, pos='v')
  gloss_vals.append(lemma)

gloss_text = " ".join(gloss_vals)
print(gloss_text)

all feel start


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [107]:
videos = pd.read_csv("/content/video-metadata.csv")

In [112]:
def get_pose_ids(gloss):
    gloss_list = gloss.split(" ")
    pose_ids = list()
    videos["Gloss Variant"] = videos["Gloss Variant"].apply(lambda x: x.strip("+#_()123456789"))
    for term in gloss_list:
        ids = videos[videos["Gloss Variant"] == term]["id"].values
        if len(ids) > 0:
            pose_ids.append(ids[0])
    return pose_ids

In [109]:
def get_pose_files(lookup_directory, pose_ids):
    pose_filepaths = []
    for pose_id in pose_ids:
        pose_filename = "pose-{}.mov".format(pose_id)
        pose_filepath = os.path.join(lookup_directory, pose_filename)
        pose_filepaths.append(pose_filepath)
    return pose_filepaths

In [114]:
pose_ids = get_pose_ids(gloss_text.upper())
gloss_available = []
for i in pose_ids:
  gloss_check = videos[videos["id"] == i]["Gloss Variant"]
  gloss_available.append(gloss_check)

gloss_available

[5    ALL
 Name: Gloss Variant, dtype: object,
 3957    FEEL
 Name: Gloss Variant, dtype: object,
 8215    START
 Name: Gloss Variant, dtype: object]

In [117]:
#upload the dataset to google drive and mount it
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#untar the file
!tar -xvf "/content/drive/MyDrive/gloss2pose-lookup.tar.gz" -C "/content/"

Streaming output truncated to the last 5000 lines.
lookup/pose-4653.mov
lookup/pose-5461.mov
lookup/pose-981.mov
lookup/pose-8860.mov
lookup/pose-2799.mov
lookup/pose-8464.mov
lookup/pose-3830.mov
lookup/pose-8619.mov
lookup/pose-9573.mov
lookup/pose-3415.mov
lookup/pose-5281.mov
lookup/pose-1501.mov
lookup/pose-1470.mov
lookup/pose-1349.mov
lookup/pose-9690.mov
lookup/pose-6598.mov
lookup/pose-7356.mov
lookup/pose-3861.mov
lookup/pose-9740.mov
lookup/pose-1457.mov
lookup/pose-7499.mov
lookup/pose-3941.mov
lookup/pose-5342.mov
lookup/pose-4975.mov
lookup/pose-3947.mov
lookup/pose-1519.mov
lookup/pose-3342.mov
lookup/pose-9069.mov
lookup/pose-2437.mov
lookup/pose-4124.mov
lookup/pose-1261.mov
lookup/pose-5424.mov
lookup/pose-9332.mov
lookup/pose-1583.mov
lookup/pose-9522.mov
lookup/pose-465.mov
lookup/pose-3913.mov
lookup/pose-9722.mov
lookup/pose-501.mov
lookup/pose-2984.mov
lookup/pose-1072.mov
lookup/pose-7188.mov
lookup/pose-2931.mov
lookup/pose-1524.mov
lookup/pose-6693.mov
lookup/

In [118]:
import os

In [119]:
video_filenames = get_pose_files('lookup/', pose_ids)

In [126]:
# creating a .mp4 file from .mov
import os.path
from os import path
video_filenames_mp4 = []
for every_video in video_filenames:
  input_video = every_video
  output_video = '/content/lookup_gloss_videolist/lookup_mp4/'+input_video.split('/')[1].split('.')[0]+'.mp4'
  video_filenames_mp4.append('/content/lookup_gloss_videolist/lookup_mp4/'+input_video.split('/')[1].split('.')[0]+'.mp4')
  if path.exists(output_video) == False:
    print("convertion to mp4 is required for: ",input_video)
    !ffmpeg -i $input_video -b:v 1M -vcodec libx264 -preset slow -profile:v baseline -level 3.0 -pix_fmt yuv420p -movflags +faststart $output_video


convertion to mp4 is required for:  lookup/pose-8215.mov
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --en

In [127]:
# Creating the content to merge the required video files
updated_gloss = gloss_text.upper().replace(" ","_")
gloss_videolist = "/content/lookup_gloss_videolist/"+updated_gloss+".txt"
gloss_videolist_to_write = "file '/content/lookup_gloss_videolist/"+updated_gloss+".txt'"
if path.exists(gloss_videolist) == False:
   with open(gloss_videolist, "w") as f:
      # Write each value in the list as a new line in the file
      for item in video_filenames_mp4:
        f.write("file " + item + "\n")


In [128]:
with open(gloss_videolist, "r") as f:
    file_content = f.read()
    print(file_content)

file /content/lookup_gloss_videolist/lookup_mp4/pose-5.mp4
file /content/lookup_gloss_videolist/lookup_mp4/pose-3957.mp4
file /content/lookup_gloss_videolist/lookup_mp4/pose-8215.mp4



In [124]:
print(updated_gloss)

ALL_FEEL_START


In [129]:
#Merging all the relavant videos
!ffmpeg -safe 0 -f concat -i "/content/lookup_gloss_videolist/ALL_FEEL_START.txt" -c copy "/content/lookup_gloss_videolist/ALL_FEEL_START.mp4"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab